# Import necessary dependencies and settings

In [8]:
import pandas as pd
import numpy as np
import string
import logging
import re
import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

from sklearn.linear_model import  LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper

import nltk
from nltk.corpus import stopwords

#import python scripts to use user_defined functions
import sys
import os
from pyprojroot import here
sys.path.append(os.path.join(here(), 'src'))

from cleaning import process_text
from training import extract_features, train_model, extract_final_features, train_final_model  
from prediction import get_predictions

# Loading the Data

In [2]:
train_news = pd.read_csv('../data/processed/train.csv').drop('len', axis=1)
val_news = pd.read_csv('../data/processed/val.csv')
test_news = pd.read_csv('../data/processed/test.csv').drop('len', axis=1)

In [3]:
train_news.head()

,label,statement
0,False,Says the Annies List political group supports ...
1,True,When did the decline of coal start? It started...
2,True,"Hillary Clinton agrees with John McCain ""by vo..."
3,False,Health care reform legislation is likely to ma...
4,True,The economic turnaround started at the end of ...


In [4]:
display(train_news), display(val_news), display(test_news)

,label,statement
0,False,Says the Annies List political group supports ...
1,True,When did the decline of coal start? It started...
2,True,"Hillary Clinton agrees with John McCain ""by vo..."
3,False,Health care reform legislation is likely to ma...
4,True,The economic turnaround started at the end of ...
...,...,...
10231,True,There are a larger number of shark attacks in ...
10232,True,Democrats have now become the party of the [At...
10233,True,Says an alternative to Social Security that op...
10234,False,On lifting the U.S. Cuban embargo and allowing...


,label,statement
0,False,We have less Americans working now than in the...
1,False,"When Obama was sworn into office, he DID NOT u..."
2,False,Says Having organizations parading as being so...
3,True,Says nearly half of Oregons children are poor.
4,True,On attacks by Republicans that various program...
...,...,...
1279,True,"For the first time in more than a decade, impo..."
1280,True,Says Donald Trump has bankrupted his companies...
1281,True,"John McCain and George Bush have ""absolutely n..."
1282,False,A new poll shows 62 percent support the presid...


,label,statement
0,True,Building a wall on the U.S.-Mexico border will...
1,False,Wisconsin is on pace to double the number of l...
2,False,Says John McCain has done nothing to help the ...
3,True,Suzanne Bonamici supports a plan that will cut...
4,False,When asked by a reporter whether hes at the ce...
...,...,...
1260,True,Says his budget provides the highest state fun...
1261,False,Ive been here almost every day.
1262,False,"In the early 1980s, Sen. Edward Kennedy secret..."
1263,False,Says an EPA permit languished under Strickland...


(None, None, None)

## Merging train & val data for K-Fold

In [5]:
"""
Merging the training and validation data together, so that I can peroform k-fold cross validation 
and shuffle the data to reduce the bias.
"""
labelEncoder = LabelEncoder()
frames = [train_news, val_news]
train_val = pd.concat(frames)
train_val['label'].value_counts()
train_val['label'] = labelEncoder.fit_transform(train_val['label'])

In [6]:
train_val

,label,statement
0,0,Says the Annies List political group supports ...
1,1,When did the decline of coal start? It started...
2,1,"Hillary Clinton agrees with John McCain ""by vo..."
3,0,Health care reform legislation is likely to ma...
4,1,The economic turnaround started at the end of ...
...,...,...
1279,1,"For the first time in more than a decade, impo..."
1280,1,Says Donald Trump has bankrupted his companies...
1281,1,"John McCain and George Bush have ""absolutely n..."
1282,0,A new poll shows 62 percent support the presid...


## Feature Weighting

Not all words are equally important to a particular document / category. For example, while words like ‘murder’, ‘knife’ and ‘abduction’ are important to a crime related document, words like ‘news’ and ‘reporter’ may not be quite as important. 

### Binary Weighting
The most basic form of feature weighting, is binary weighting. Where if a word is present in a document, the weight is ‘1’ and if the word is absent the weight is ‘0’. 

### CountVectorizer

It Convert a collection of text documents to a matrix of token counts.


### Tfidf Weighting 

TF-IDF weighting where words that are unique to a particular document would have higher weights compared to words that are used commonly across documents. 

1. TF (Term Frequency): The number of times a word appears in a document divded by the total number of words in the document. Every document has its own term frequency.

2. IDF (Inverse Data Frequency): The log of the number of documents divided by the number of documents that contain the word w. Inverse data frequency determines the weight of rare words across all documents in the corpus.

3. Lastly, the TF-IDF is simply the TF multiplied by IDF.

## Metric

I need to minimize false positives (number of fake news predicted as real news) as it can be very misleadling . For class 0 i.e. 'fake', recall should be high as well as precision. Because we want our model to perform well on both classes (real & fake). In short, we need to maximize f1-score.

### Cases I considered to choose the right metric

**1. Maximizing recall of class 0 (fake) or minimizing false positives(FP)?**
Well, in extreme case, what if all the news predicted by model are labelled as 'fake'. Recall will still be 1, but overall model is really bad i.e. not able to predict class 1 ('real'). 

Ex=> TN = 553, FP = 0, TP = 0, FN = 714

Class0-Recall = TN / (TN + FP) = 1
Class0-Precision = TN / (TN + FN) = 0.43

F1-Score = 2 * Class0-Recall * Class0-Precision/(Class0-Recall + Class0-Precision) = 0.60

Recall, Precision and F1-score for class 1 will be 0.

**2. Considering an extreme case, if all the news classified as True (Even, fake news are predicted as True).**

Ex=>  TN = 0, FP = 553, TP = 714, FN =0
In that case, TN will be 0, which led to Precision 0, Recall 0 and F1 = 0 for class 0 ('fake').

For class 1, Class1-Recall = TP / (TP + FN) = 1
Class1-Precision = TP / (TP + FP) = 0.56

## Model Training

## Text Classification Algorithms

1. Naive Bayes (NB)
2. Logistics Regression
3. SVM
4. Random Forest

## Naive Bayes

Well, when assumption of independence holds, a Naive Bayes classifier performs better compare to other models like logistic regression and you need less training data. An advantage of naive Bayes is that it only requires a small number of training data to estimate the parameters necessary for classification. 

Bayes’ Theorem provides a way that we can calculate the probability of a piece of data belonging to a given class, given our prior knowledge. Bayes’ Theorem is stated as:

P(class|data) = (P(data|class) * P(class)) / P(data)

Where P(class|data) is the probability of class given the provided data.

Naive Bayes is a classification algorithm for binary (two-class) and multiclass classification problems. It is called Naive Bayes or idiot Bayes because the calculations of the probabilities for each class are simplified to make their calculations tractable.

Rather than attempting to calculate the probabilities of each attribute value, they are assumed to be conditionally independent given the class value.

This is a very strong assumption that is most unlikely in real data, i.e. that the attributes do not interact. Nevertheless, the approach performs surprisingly well on data where this assumption does not hold.

### Multinomial NB

The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work

### Train Models with Different Types of Features

In [7]:
# model,transformer,score,confusion,report=train_model(nb_clf, train_val,field=field,feature_rep=feature_rep)
# print("\nF1-score={0}; confusion={1}; classification_report={2}".format(score,confusion,report))
field='statement'
feature_reps=['binary','counts','tfidf']
nb_results=[]
nb_clf = MultinomialNB()
for feature_rep in feature_reps:
        print(f'Model - {feature_rep} features with statement')
        nb_model,transformer,score=train_model(nb_clf,train_val,field=field,feature_rep=feature_rep)
        nb_results.append([field,feature_rep,score])

2020-12-04 21:31:10,413 : INFO : Starting model training...
2020-12-04 21:31:10,420 : INFO : Extracting features and creating vocabulary...


Model - binary features with statement


2020-12-04 21:31:50,295 : INFO : Training a Classification Model...
2020-12-04 21:31:50,299 : INFO : Starting evaluation...
2020-12-04 21:31:50,309 : INFO : Done training and evaluation.
2020-12-04 21:31:50,311 : INFO : Starting model training...
2020-12-04 21:31:50,314 : INFO : Extracting features and creating vocabulary...


              precision    recall  f1-score   support

           0       0.55      0.48      0.51      1248
           1       0.64      0.70      0.67      1632

    accuracy                           0.60      2880
   macro avg       0.59      0.59      0.59      2880
weighted avg       0.60      0.60      0.60      2880

[[ 596  652]
 [ 488 1144]]
Model - counts features with statement


2020-12-04 21:32:28,344 : INFO : Training a Classification Model...
2020-12-04 21:32:28,349 : INFO : Starting evaluation...
2020-12-04 21:32:28,360 : INFO : Done training and evaluation.
2020-12-04 21:32:28,362 : INFO : Starting model training...
2020-12-04 21:32:28,367 : INFO : Extracting features and creating vocabulary...


              precision    recall  f1-score   support

           0       0.55      0.48      0.52      1248
           1       0.64      0.70      0.67      1632

    accuracy                           0.61      2880
   macro avg       0.60      0.59      0.59      2880
weighted avg       0.60      0.61      0.60      2880

[[ 602  646]
 [ 485 1147]]
Model - tfidf features with statement


2020-12-04 21:33:06,207 : INFO : Training a Classification Model...
2020-12-04 21:33:06,211 : INFO : Starting evaluation...
2020-12-04 21:33:06,221 : INFO : Done training and evaluation.


              precision    recall  f1-score   support

           0       0.58      0.31      0.41      1248
           1       0.61      0.83      0.70      1632

    accuracy                           0.60      2880
   macro avg       0.60      0.57      0.56      2880
weighted avg       0.60      0.60      0.58      2880

[[ 391  857]
 [ 281 1351]]


### Naive Bayes Results of Various Models

In [8]:
nb_df_results=pd.DataFrame(nb_results,columns=['text_fields','feature_representation','f1-score'])
nb_df_results.sort_values(by=['f1-score'],ascending=False)

,text_fields,feature_representation,f1-score
2,statement,tfidf,0.703646
1,statement,counts,0.669781
0,statement,binary,0.667445


## logistic regression

The underlying algorithm is also fairly easy to understand. More importantly, in the NLP world, it’s generally accepted that Logistic Regression is a great starter algorithm for text related classification (https://web.stanford.edu/~jurafsky/slp3/5.pdf). 

**How hypothesis makes prediction in logistics regression?**

This algorithm uses sigmoid function(g(z)). If we want to predict y=1 or y=0.
If estimated probability of y=1 is h(x)>=0.5 then the ouput is more likely to be "y=1" 
but if  h(x) < 0.5, the output is more likely to be is "y=0".

### Train Models with Different Types of Features¶

In [21]:
field='statement'
feature_reps=['binary','counts','tfidf']
lr_results=[]
LogR_clf = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)

for feature_rep in feature_reps:
        print(f'Model - {feature_rep} features with statement')
        lr_model,transformer,score=train_model(LogR_clf,train_val,field=field,feature_rep=feature_rep)
        lr_results.append([field,feature_rep,score])

2020-12-03 20:44:04,794 : INFO : Starting model training...
2020-12-03 20:44:04,800 : INFO : Extracting features and creating vocabulary...


Model - binary features with statement


2020-12-03 20:44:46,314 : INFO : Training a Classification Model...
2020-12-03 20:44:46,400 : INFO : Starting evaluation...
2020-12-03 20:44:46,412 : INFO : Done training and evaluation.
2020-12-03 20:44:46,413 : INFO : Starting model training...
2020-12-03 20:44:46,418 : INFO : Extracting features and creating vocabulary...


[LibLinear]              precision    recall  f1-score   support

           0       0.51      0.51      0.51      1248
           1       0.62      0.62      0.62      1632

    accuracy                           0.57      2880
   macro avg       0.57      0.57      0.57      2880
weighted avg       0.57      0.57      0.57      2880

[[ 636  612]
 [ 617 1015]]
Model - counts features with statement


2020-12-03 20:45:29,889 : INFO : Training a Classification Model...
2020-12-03 20:45:29,997 : INFO : Starting evaluation...
2020-12-03 20:45:30,012 : INFO : Done training and evaluation.
2020-12-03 20:45:30,014 : INFO : Starting model training...
2020-12-03 20:45:30,027 : INFO : Extracting features and creating vocabulary...


[LibLinear]              precision    recall  f1-score   support

           0       0.50      0.50      0.50      1248
           1       0.62      0.62      0.62      1632

    accuracy                           0.57      2880
   macro avg       0.56      0.56      0.56      2880
weighted avg       0.57      0.57      0.57      2880

[[ 620  628]
 [ 612 1020]]
Model - tfidf features with statement


2020-12-03 20:46:11,374 : INFO : Training a Classification Model...
2020-12-03 20:46:11,419 : INFO : Starting evaluation...
2020-12-03 20:46:11,434 : INFO : Done training and evaluation.


[LibLinear]              precision    recall  f1-score   support

           0       0.53      0.49      0.51      1248
           1       0.63      0.66      0.65      1632

    accuracy                           0.59      2880
   macro avg       0.58      0.58      0.58      2880
weighted avg       0.59      0.59      0.59      2880

[[ 615  633]
 [ 552 1080]]


### Logistics Regression Results of Various Models

In [22]:
lr_df_results=pd.DataFrame(lr_results,columns=['text_fields','feature_representation','f1-score'])
lr_df_results.sort_values(by=['f1-score'],ascending=False)

,text_fields,feature_representation,f1-score
2,statement,tfidf,0.645740
0,statement,binary,0.622890
1,statement,counts,0.621951


Here you see how the performance of logistics model is improved using tfidf over counts and binary weightning.

## SVM

Support vector machines is an algorithm that determines the best decision boundary between vectors that belong to a given group (or category) and vectors that do not belong to it. That’s it. It can be applied to any kind of vectors which encode any kind of data. This means that in order to leverage the power of svm text classification, texts have to be transformed into vectors.

So, when SVM determines the decision boundary we mentioned above, SVM decides where to draw the best “line” (or the best hyperplane) that divides the space into two subspaces: one for the vectors which belong to the given category and one for the vectors which do not belong to it.

### Train Models with Different Types of Features¶

In [23]:
field='statement'
feature_reps=['binary','counts','tfidf']
svm_results=[]
svm_clf = svm.LinearSVC()

for feature_rep in feature_reps:
        print(f'SVM Model - {feature_rep} features with statement')
        svm_model,transformer,score=train_model(svm_clf,train_val,field=field,feature_rep=feature_rep)
        svm_results.append([field,feature_rep,score])

2020-12-03 20:46:11,467 : INFO : Starting model training...
2020-12-03 20:46:11,474 : INFO : Extracting features and creating vocabulary...


SVM Model - binary features with statement


2020-12-03 20:46:49,509 : INFO : Training a Classification Model...
2020-12-03 20:46:49,651 : INFO : Starting evaluation...
2020-12-03 20:46:49,661 : INFO : Done training and evaluation.
2020-12-03 20:46:49,663 : INFO : Starting model training...
2020-12-03 20:46:49,667 : INFO : Extracting features and creating vocabulary...


              precision    recall  f1-score   support

           0       0.50      0.50      0.50      1248
           1       0.61      0.61      0.61      1632

    accuracy                           0.56      2880
   macro avg       0.55      0.55      0.55      2880
weighted avg       0.56      0.56      0.56      2880

[[622 626]
 [634 998]]
SVM Model - counts features with statement


2020-12-03 20:47:27,984 : INFO : Training a Classification Model...
2020-12-03 20:47:28,175 : INFO : Starting evaluation...
2020-12-03 20:47:28,185 : INFO : Done training and evaluation.
2020-12-03 20:47:28,187 : INFO : Starting model training...
2020-12-03 20:47:28,192 : INFO : Extracting features and creating vocabulary...


              precision    recall  f1-score   support

           0       0.49      0.50      0.50      1248
           1       0.61      0.61      0.61      1632

    accuracy                           0.56      2880
   macro avg       0.55      0.55      0.55      2880
weighted avg       0.56      0.56      0.56      2880

[[622 626]
 [635 997]]
SVM Model - tfidf features with statement


2020-12-03 20:48:06,256 : INFO : Training a Classification Model...
2020-12-03 20:48:06,299 : INFO : Starting evaluation...
2020-12-03 20:48:06,312 : INFO : Done training and evaluation.


              precision    recall  f1-score   support

           0       0.52      0.50      0.51      1248
           1       0.63      0.65      0.64      1632

    accuracy                           0.58      2880
   macro avg       0.57      0.57      0.57      2880
weighted avg       0.58      0.58      0.58      2880

[[ 622  626]
 [ 572 1060]]


### SVM Results of Various Models

In [24]:
svm_df_results=pd.DataFrame(svm_results,columns=['text_fields','feature_representation','f1-score'])
svm_df_results.sort_values(by=['f1-score'],ascending=False)

,text_fields,feature_representation,f1-score
2,statement,tfidf,0.638939
0,statement,binary,0.613022
1,statement,counts,0.612596


## Random Forest

Given the nature of random forests (a bagging decision tree), it is true that you may come up with a rather weak classifier, especially if only a couple of features are truly significant to determine the outcome.

However, keep in mind that in the case of text classification, a preprocessing phase is required to get either your TF or TF-IDF matrix, through which you have already made a selection of pertinent features. Potentially, all features are relevant in this matrix, so the random forest may be performant when you predict your outcome. (source: https://stats.stackexchange.com/questions/343954/random-forest-short-text-classification)

### Train Models with Different Types of Features¶

In [25]:
field='statement'
feature_reps=['binary','counts','tfidf']
rf_results=[]
rf_clf = RandomForestClassifier(n_estimators=1000)

for feature_rep in feature_reps:
        rf_model,transformer,score=train_model(rf_clf,train_val,field=field,feature_rep=feature_rep)
        rf_results.append([field,feature_rep,score])

2020-12-03 20:48:06,348 : INFO : Starting model training...
2020-12-03 20:48:06,353 : INFO : Extracting features and creating vocabulary...
2020-12-03 20:48:52,329 : INFO : Training a Classification Model...
2020-12-03 20:50:26,183 : INFO : Starting evaluation...
2020-12-03 20:50:26,193 : INFO : Done training and evaluation.
2020-12-03 20:50:26,196 : INFO : Starting model training...
2020-12-03 20:50:26,200 : INFO : Extracting features and creating vocabulary...


              precision    recall  f1-score   support

           0       0.56      0.45      0.50      1248
           1       0.63      0.73      0.68      1632

    accuracy                           0.61      2880
   macro avg       0.60      0.59      0.59      2880
weighted avg       0.60      0.61      0.60      2880

[[ 561  687]
 [ 443 1189]]


2020-12-03 20:51:11,303 : INFO : Training a Classification Model...
2020-12-03 20:52:24,718 : INFO : Starting evaluation...
2020-12-03 20:52:24,729 : INFO : Done training and evaluation.
2020-12-03 20:52:24,732 : INFO : Starting model training...
2020-12-03 20:52:24,735 : INFO : Extracting features and creating vocabulary...


              precision    recall  f1-score   support

           0       0.55      0.43      0.49      1248
           1       0.63      0.73      0.68      1632

    accuracy                           0.60      2880
   macro avg       0.59      0.58      0.58      2880
weighted avg       0.59      0.60      0.59      2880

[[ 540  708]
 [ 438 1194]]


2020-12-03 20:53:02,968 : INFO : Training a Classification Model...
2020-12-03 20:54:09,725 : INFO : Starting evaluation...
2020-12-03 20:54:09,735 : INFO : Done training and evaluation.


              precision    recall  f1-score   support

           0       0.55      0.42      0.48      1248
           1       0.63      0.74      0.68      1632

    accuracy                           0.60      2880
   macro avg       0.59      0.58      0.58      2880
weighted avg       0.59      0.60      0.59      2880

[[ 528  720]
 [ 428 1204]]


### RF Results of Various Models¶

In [26]:
rf_df_results=pd.DataFrame(rf_results,columns=['text_fields','feature_representation','f1-score'])
rf_df_results.sort_values(by=['f1-score'],ascending=False)

,text_fields,feature_representation,f1-score
0,statement,binary,0.677879
2,statement,tfidf,0.677165
1,statement,counts,0.675722


## K-fold cross validation

With K-fold cross validation, you are testing how well your model is able to get trained by some data and then predict data it hasn't seen. We use cross validation for this because if you train using all the data you have, you have none left for testing. You could do this once, say by using 80% of the data to train and 20% to test, but what if the 20% you happened to pick to test happens to contain a bunch of points that are particularly easy (or particularly hard) to predict? We will not have come up with the best estimate possible of the models ability to learn and predict.

In [30]:
#User defined functon for K-Fold cross validatoin
def apply_kfold(classifier,train_val,field,feature_rep):
    """
    K-fold cross validation on the the data
    """
    k_fold = KFold(n_splits=5, shuffle=True)
    scores = []
    confusion = np.array([[0,0],[0,0]])

    for fold_n, (train_index, valid_index) in enumerate(k_fold.split(train_val['statement'], train_val['label'])):
        print(fold_n, len(train_index), len(valid_index))
        train_x = train_val['statement'].iloc[train_index]
        train_y = train_val['label'].iloc[train_index]
    
        valid_x = train_val['statement'].iloc[valid_index]
        valid_y = train_val['label'].iloc[valid_index]
        
        # GET FEATURES
        train_features,val_features,feature_transformer=extract_features(field,train_x,valid_x,type=feature_rep)
        
        # INIT CLASSIFIER
        logging.info("Training a Classification Model...")
        classifier.fit(train_features, train_y)
        predictions = classifier.predict(val_features)
        
        confusion += confusion_matrix(valid_y,predictions)
        score = f1_score(valid_y,predictions)
        scores.append(score)
        
    return (print('Total statements classified:', len(train_val['statement'])),
    print('Score:', sum(scores)/len(scores)),
    print('score length', len(scores)),
    print('Confusion matrix:'),
    print(confusion))

## Grid Search Hyperparameters with K-Fold Cross Validation

## Logistics Regression

In [14]:
grid={"C":np.logspace(-3,3,7), "penalty":["l2"], "max_iter":[500, 1000]}# l1 lasso l2 ridge
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=5)

# GET FEATURES
feature_reps=['binary','counts','tfidf']
train_features,feature_transformer=extract_final_features(field,train_val['statement'],type='binary')
logreg_cv.fit(train_features,train_val['label'])

2020-12-04 14:27:12,473 : INFO : Extracting features and creating vocabulary...
/opt/anaconda3/envs/newenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/envs/newenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative sol

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'max_iter': [500, 1000], 'penalty': ['l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [15]:
print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

tuned hpyerparameters :(best parameters)  {'C': 0.1, 'max_iter': 500, 'penalty': 'l2'}
accuracy : 0.6151041666666667


## SVC

In [16]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from skopt import BayesSearchCV

field='statement'
feature_reps=['binary','counts','tfidf']
# GET FEATURES
train_features,feature_transformer=extract_final_features('statement',train_val['statement'],type='binary')
    
# define search space
params = dict()
params['C'] = (1e-6, 100.0, 'log-uniform')
params['gamma'] = (1e-6, 100.0, 'log-uniform')
params['degree'] = (1,5)
params['kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']
# define evaluation
cv = StratifiedKFold(n_splits=5, random_state=1)
# define the search
search = BayesSearchCV(estimator=SVC(), search_spaces=params, n_jobs=-1, cv=cv)
# perform the search
search.fit(train_features, train_val['label'])
# report the best result
print(search.best_score_)
print(search.best_params_)

2020-12-04 14:38:28,037 : INFO : Extracting features and creating vocabulary...
/opt/anaconda3/envs/newenv/lib/python3.8/site-packages/sklearn/model_selection/_split.py:292: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


0.6172743055555555
OrderedDict([('C', 0.00020805609531827864), ('degree', 1), ('gamma', 100.0), ('kernel', 'poly')])


## Random Forest

In [20]:
rfc = RandomForestClassifier(n_jobs=-1,oob_score = True) 

param_grid = { 
    'n_estimators': [200, 700],
    'max_features': ['sqrt', 'log2']
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
# get features
train_features,feature_transformer=extract_final_features('statement',train_val['statement'],type='binary')
CV_rfc.fit(train_features, train_val['label'])
print(CV_rfc.best_params_)

2020-12-04 15:44:02,202 : INFO : Extracting features and creating vocabulary...


{'max_features': 'log2', 'n_estimators': 700}


In [25]:
print(CV_rfc.best_score_)

0.6276041666666666


## Naive Bayes with K-fold cross validation

In [32]:
field='statement'
feature_reps=['binary','counts','tfidf']
nb_results=[]
nb_clf = MultinomialNB()
for feature_rep in feature_reps:
        print(f'Model - {feature_rep} features with statement')
        apply_kfold(nb_clf,train_val,field=field,feature_rep=feature_rep)

2020-12-03 20:54:40,953 : INFO : Extracting features and creating vocabulary...


Model - binary features with statement
0 9216 2304


2020-12-03 20:55:23,804 : INFO : Training a Classification Model...
2020-12-03 20:55:23,814 : INFO : Extracting features and creating vocabulary...


1 9216 2304


2020-12-03 20:56:03,322 : INFO : Training a Classification Model...
2020-12-03 20:56:03,331 : INFO : Extracting features and creating vocabulary...


...

3 9216 2304


2020-12-03 21:03:53,362 : INFO : Training a Classification Model...
2020-12-03 21:03:53,370 : INFO : Extracting features and creating vocabulary...


4 9216 2304


2020-12-03 21:04:32,660 : INFO : Training a Classification Model...


Total statements classified: 11520
Score: 0.7032826381843794
score length 5
Confusion matrix:
[[1477 3625]
 [ 971 5447]]


## Logistics Regression with K-fold cross Validation

In [17]:
# field='statement'
# feature_reps=['binary','counts','tfidf']
# LogR_clf = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)

# for feature_rep in feature_reps:
#         print(f'Model - {feature_rep} features with statement')
#         apply_kfold(LogR_clf,train_val,field=field,feature_rep=feature_rep)

## SVM with K-fold cross Validation

In [18]:
# field='statement'
# feature_reps=['binary','counts','tfidf']
# svm_clf = svm.LinearSVC()

# for feature_rep in feature_reps:
#         print(f'Model - {feature_rep} features with statement')
#         apply_kfold(svm_clf,train_val,field=field,feature_rep=feature_rep)

## RF with K-fold cross Validation

In [19]:
# field='statement'
# feature_reps=['binary','counts','tfidf']
# rf_clf = RandomForestClassifier(n_estimators=1000)

# for feature_rep in feature_reps:
#         print(f'Model - {feature_rep} features with statement')
#         apply_kfold(rf_clf,train_val,field=field,feature_rep=feature_rep)

## Best Model Selection

"""
Out of all the models fitted, we would take 2 best performing model. we would call them candidate models
from the confusion matrix, we can see that logistic regression and SVM (with either binary or tfidf features) are better performing 
in terms of precision and recall (take a look into false positive and true negative counts which appeares
to be low compared to rest of the models).

Using k-fold cross validation, we see the performance of the models on the entire dataset. And, the model's aren't performing well. We can apply other features to improve the performance, and grid-search can also help us to find best parameters to improve the perfromance.
"""

In [9]:
field='statement'
LogR_clf_final = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
lr_final_model,transformer=train_final_model(LogR_clf_final,train_val,field=field,feature_rep='counts')

2020-12-05 13:52:48,913 : INFO : Starting model training...
2020-12-05 13:52:48,925 : INFO : Extracting features and creating vocabulary...
2020-12-05 13:54:26,774 : INFO : Training a Final Model...


[LibLinear]

2020-12-05 13:54:27,074 : INFO : Done training.


In [47]:
# train_val['statement'].values.values

## Saving Model

In [10]:
model_path="../models/lr_final_model.pkl"
transformer_path="../models/transformer.pkl"

In [43]:
# we need to save both the transformer -> to encode a document and the model itself to make predictions based on the weight vectors 
pickle.dump(lr_final_model,open(model_path, 'wb'))
pickle.dump(transformer,open(transformer_path,'wb'))

## Loading model 

In [11]:
# load the model and feature transformer with pickle
loaded_model = pickle.load(open(model_path, 'rb'))
loaded_transformer = pickle.load(open(transformer_path, 'rb'))

## Check predictions on unseen data

In [17]:
input_text = process_text("When asked by a reporter whether hes at the center of a criminal scheme to violate campaign laws, Gov. Scott Walker nodded yes.")
test_features = loaded_transformer.transform([" ".join(input_text)])
output = get_predictions(loaded_model,test_features)
output[0] # correctly predicted

0

In [18]:
input_text = process_text("Says John McCain has done nothing to help the vets.")
test_features=loaded_transformer.transform([" ".join(input_text)])
ouput = get_predictions(loaded_model,test_features)
ouput[0] # false news predicted as false. #correct prediction

0

In [19]:
input_text = process_text("Says that Tennessee law requires that schools receive half of proceeds -- $31 million per year -- from a half-cent increase in the Shelby County sales tax.")
test_features=loaded_transformer.transform([" ".join(input_text)])
ouput = get_predictions(loaded_model,test_features)
ouput[0] # True news predicted as True. #correct prediction

0